This script is for converting the matsim output files (generated by Aurore) into osm files for R5

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import geopandas as gpd
import shapely as shp

import snman
from snman.constants import *
from snman import osmnx_customized as oxc

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join(
    'C:',os.sep,'Users','lballo','polybox','Research',
    'SNMan','SNMan Shared','data_v2'
)
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)
paper_path = os.path.join(
    'C:',os.sep,'Users','lballo','polybox','Research',
    'E-Bike City Accessibility','EBC Accessibility Paper - Shared'
)
matsim_results_path = os.path.join(
    paper_path, 'MATSim results', '2024-07-12 Travel times before and after'
)

#CRS_internal = 29119    # for Boston
#CRS_internal = 32216    # for Chicago
CRS_internal = 2056      # for Zurich
CRS_for_export = 4326
oxc.settings.useful_tags_way = OSM_TAGS

In [2]:
TIME = '18:00'
STATE = 'after'

In [3]:
tt = pd.read_csv(
    os.path.join(matsim_results_path, f'{STATE}_bike100pct.csv')
)

#tt

,LinkId,OSM_ID,FreeflowTTCar,FreeFlowTTBike,From_x,From_y,To_x,To_y,car_0:00,bike_0:00,...,bike_21:30,car_22:00,bike_22:00,car_22:30,bike_22:30,car_23:00,bike_23:00,car_23:30,bike_23:30,Unnamed: 104
0,3640,7701054,22.34,44.68,47.523778,8.792639,47.522871,8.794674,22.34,44.68,...,44.68,28.27,44.68,27.10,44.68,29.00,44.68,24.62,44.68,NaN
1,3638,7701052,13.91,27.81,47.523778,8.792639,47.524240,8.791260,13.91,27.81,...,31.00,20.00,27.81,20.00,27.81,13.91,27.81,20.00,27.81,NaN
2,3639,7701053,9.81,19.62,47.523778,8.792639,47.523084,8.792281,9.81,19.62,...,19.62,16.00,19.62,16.00,19.62,16.00,19.62,16.00,19.62,NaN
3,3630,7715687,5.86,11.73,47.534438,8.761990,47.534527,8.761355,5.86,11.73,...,16.00,12.00,15.00,12.00,15.00,12.00,15.00,12.00,15.00,NaN
4,3631,7715688,4.39,8.79,47.534856,8.761385,47.534527,8.761355,4.39,8.79,...,8.79,11.00,8.79,11.00,8.79,11.00,12.00,11.00,8.79,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143330,3629,7715686,9.62,19.24,47.534438,8.761990,47.534296,8.763034,9.62,19.24,...,23.00,14.75,19.24,14.06,23.00,9.62,23.00,16.00,19.24,NaN
143331,75829,7591240,3.14,10.48,47.276250,8.755897,47.276335,8.756461,3.14,10.48,...,14.00,10.00,14.00,10.00,10.48,10.00,14.00,10.00,10.48,NaN
143332,3628,7701043,17.95,35.91,47.522260,8.796916,47.523033,8.798388,17.95,35.91,...,35.91,17.95,35.91,17.95,35.91,17.95,35.91,18.00,35.91,NaN
143333,3627,7701042,11.98,23.97,47.522260,8.796916,47.521467,8.797539,11.98,23.97,...,23.97,12.00,23.97,11.98,23.97,11.98,23.97,11.98,23.97,NaN


In [4]:
import pyproj

transformer = pyproj.Transformer.from_crs(4326, 2056, always_xy=True)

tt['u'] = tt.apply(
    lambda row: shp.Point(
        *transformer.transform(row['From_y'], row['From_x'])
    ),
    axis=1
)
tt['v'] = tt.apply(
    lambda row: shp.Point(
        *transformer.transform(row['To_y'], row['To_x'])
    ),
    axis=1
)

tt['geometry'] = tt.apply(
    lambda row: shp.LineString([row['u'], row['v']]),
    axis=1
)

tt = gpd.GeoDataFrame(tt, crs=2056)

tt

,LinkId,OSM_ID,FreeflowTTCar,FreeFlowTTBike,From_x,From_y,To_x,To_y,car_0:00,bike_0:00,...,car_22:30,bike_22:30,car_23:00,bike_23:00,car_23:30,bike_23:30,Unnamed: 104,u,v,geometry
0,3640,7701054,22.34,44.68,47.523778,8.792639,47.522871,8.794674,22.34,44.68,...,27.10,44.68,29.00,44.68,24.62,44.68,NaN,POINT (2701971.2509862264 1264550.459269584),POINT (2702126.196330599 1264452.3290083027),"LINESTRING (2701971.251 1264550.459, 2702126.1..."
1,3638,7701052,13.91,27.81,47.523778,8.792639,47.524240,8.791260,13.91,27.81,...,20.00,27.81,13.91,27.81,20.00,27.81,NaN,POINT (2701971.2509862264 1264550.459269584),POINT (2701866.522171238 1264600.073411381),"LINESTRING (2701971.251 1264550.459, 2701866.5..."
2,3639,7701053,9.81,19.62,47.523778,8.792639,47.523084,8.792281,9.81,19.62,...,16.00,19.62,16.00,19.62,16.00,19.62,NaN,POINT (2701971.2509862264 1264550.459269584),POINT (2701945.5899760537 1264472.8407860398),"LINESTRING (2701971.251 1264550.459, 2701945.5..."
3,3630,7715687,5.86,11.73,47.534438,8.761990,47.534527,8.761355,5.86,11.73,...,12.00,15.00,12.00,15.00,12.00,15.00,NaN,POINT (2699643.28363578 1265696.1912974257),POINT (2699595.285331079 1265705.3345726437),"LINESTRING (2699643.284 1265696.191, 2699595.2..."
4,3631,7715688,4.39,8.79,47.534856,8.761385,47.534527,8.761355,4.39,8.79,...,11.00,8.79,11.00,12.00,11.00,8.79,NaN,POINT (2699596.957955538 1265741.9154586608),POINT (2699595.285331079 1265705.3345726437),"LINESTRING (2699596.958 1265741.915, 2699595.2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143330,3629,7715686,9.62,19.24,47.534438,8.761990,47.534296,8.763034,9.62,19.24,...,14.06,23.00,9.62,23.00,16.00,19.24,NaN,POINT (2699643.28363578 1265696.1912974257),POINT (2699722.1511149006 1265681.7201363703),"LINESTRING (2699643.284 1265696.191, 2699722.1..."
143331,75829,7591240,3.14,10.48,47.276250,8.755897,47.276335,8.756461,3.14,10.48,...,10.00,10.48,10.00,14.00,10.00,10.48,NaN,POINT (2699666.474238684 1236986.3436005437),POINT (2699708.9396957774 1236996.4749395344),"LINESTRING (2699666.474 1236986.344, 2699708.9..."
143332,3628,7701043,17.95,35.91,47.522260,8.796916,47.523033,8.798388,17.95,35.91,...,17.95,35.91,17.95,35.91,18.00,35.91,NaN,POINT (2702296.2196253994 1264387.2857025354),POINT (2702405.547817276 1264475.1905265348),"LINESTRING (2702296.220 1264387.286, 2702405.5..."
143333,3627,7701042,11.98,23.97,47.522260,8.796916,47.521467,8.797539,11.98,23.97,...,11.98,23.97,11.98,23.97,11.98,23.97,NaN,POINT (2702296.2196253994 1264387.2857025354),POINT (2702344.6761799133 1264299.9677734694),"LINESTRING (2702296.220 1264387.286, 2702344.6..."


In [5]:
nd = pd.concat([tt['u'], tt['v']]).reset_index().rename(columns={0: 'geometry'})
nd['osmid'] = pd.factorize(nd['geometry'])[0]
nd[['x', 'y']] = nd.apply(
    lambda row: (row['geometry'].x, row['geometry'].y),
    axis=1,
    result_type='expand'
)
nd.drop(columns=['index'], inplace=True)
nd.drop_duplicates(inplace=True)
nd.set_index('osmid', inplace=True)
nd = gpd.GeoDataFrame(nd, geometry='geometry', crs=2056)

nd

,geometry,x,y
osmid,,,
0,POINT (2701971.251 1264550.459),2.701971e+06,1.264550e+06
1,POINT (2699643.284 1265696.191),2.699643e+06,1.265696e+06
2,POINT (2699596.958 1265741.915),2.699597e+06,1.265742e+06
3,POINT (2702126.196 1264452.329),2.702126e+06,1.264452e+06
4,POINT (2638563.749 1242835.829),2.638564e+06,1.242836e+06
...,...,...,...
61804,POINT (2646040.731 1255217.002),2.646041e+06,1.255217e+06
61805,POINT (2671606.265 1255098.431),2.671606e+06,1.255098e+06
61806,POINT (2681171.862 1241293.762),2.681172e+06,1.241294e+06


In [6]:
tt = (pd.merge(tt, nd.reset_index()[['osmid', 'geometry']], how='left', left_on='u', right_on='geometry', suffixes=['', '_right'])
    .drop(columns=['u', 'geometry_right'])
    .rename(columns={'osmid': 'u'}))
tt = (pd.merge(tt, nd.reset_index()[['osmid', 'geometry']], how='left', left_on='v', right_on='geometry', suffixes=['', '_right'])
    .drop(columns=['v', 'geometry_right'])
    .rename(columns={'osmid': 'v'}))

tt.drop(columns=['From_x', 'From_y', 'To_x', 'To_y'], inplace=True)

tt['uv'] = tt.apply(lambda row: (row['u'], row['v']), axis=1)
tt.set_index('uv', inplace=True)

tt['length'] = tt.apply(lambda row: row.geometry.length, axis=1)
tt['travel_time_s'] = tt[f'car_{TIME}'].replace(0, 99999)
#tt['highway'] = 'primary'
tt['maxspeed'] = 40

tt

,LinkId,OSM_ID,FreeflowTTCar,FreeFlowTTBike,car_0:00,bike_0:00,car_0:30,bike_0:30,car_1:00,bike_1:00,...,bike_23:00,car_23:30,bike_23:30,Unnamed: 104,geometry,u,v,length,travel_time_s,maxspeed
uv,,,,,,,,,,,,,,,,,,,,,
"(0, 3)",3640,7701054,22.34,44.68,22.34,44.68,22.34,44.68,22.34,44.68,...,44.68,24.62,44.68,NaN,"LINESTRING (2701971.251 1264550.459, 2702126.1...",0,3,183.405583,28.46,40
"(0, 6916)",3638,7701052,13.91,27.81,13.91,27.81,13.91,27.81,13.91,27.81,...,27.81,20.00,27.81,NaN,"LINESTRING (2701971.251 1264550.459, 2701866.5...",0,6916,115.886530,20.32,40
"(0, 4321)",3639,7701053,9.81,19.62,9.81,19.62,9.81,19.62,9.81,19.62,...,19.62,16.00,19.62,NaN,"LINESTRING (2701971.251 1264550.459, 2701945.5...",0,4321,81.750330,16.06,40
"(1, 57401)",3630,7715687,5.86,11.73,5.86,11.73,5.86,11.73,5.86,11.73,...,15.00,12.00,15.00,NaN,"LINESTRING (2699643.284 1265696.191, 2699595.2...",1,57401,48.861403,12.02,40
"(2, 57401)",3631,7715688,4.39,8.79,4.39,8.79,4.39,8.79,4.39,8.79,...,12.00,11.00,8.79,NaN,"LINESTRING (2699596.958 1265741.915, 2699595.2...",2,57401,36.619106,11.00,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(1, 11923)",3629,7715686,9.62,19.24,9.62,19.24,9.62,19.24,9.62,19.24,...,23.00,16.00,19.24,NaN,"LINESTRING (2699643.284 1265696.191, 2699722.1...",1,11923,80.184124,15.59,40
"(61652, 39215)",75829,7591240,3.14,10.48,3.14,10.48,3.14,10.48,3.14,10.48,...,14.00,10.00,10.48,NaN,"LINESTRING (2699666.474 1236986.344, 2699708.9...",61652,39215,43.657291,10.02,40
"(5, 30213)",3628,7701043,17.95,35.91,17.95,35.91,17.95,35.91,17.95,35.91,...,35.91,18.00,35.91,NaN,"LINESTRING (2702296.220 1264387.286, 2702405.5...",5,30213,140.285108,18.00,40


In [7]:
#to fill in the missing highway tags, we load the original osm export
osm_export = snman.io.import_geofile_to_gdf(
    os.path.join(matsim_results_path, f'{STATE}_oneway_links.gpkg')
)

osm_export['osm_id'] = osm_export['osm_id'].astype('int64')
tt = pd.merge(
    tt.reset_index(), osm_export[['osm_id', 'highway']],
    left_on='OSM_ID', right_on='osm_id', how='left'
)
tt.drop(columns='osm_id', inplace=True)
tt.set_index('uv', inplace=True)
tt

,LinkId,OSM_ID,FreeflowTTCar,FreeFlowTTBike,car_0:00,bike_0:00,car_0:30,bike_0:30,car_1:00,bike_1:00,...,car_23:30,bike_23:30,Unnamed: 104,geometry,u,v,length,travel_time_s,maxspeed,highway
uv,,,,,,,,,,,,,,,,,,,,,
"(0, 3)",3640,7701054,22.34,44.68,22.34,44.68,22.34,44.68,22.34,44.68,...,24.62,44.68,NaN,"LINESTRING (2701971.251 1264550.459, 2702126.1...",0,3,183.405583,28.46,40,residential
"(0, 6916)",3638,7701052,13.91,27.81,13.91,27.81,13.91,27.81,13.91,27.81,...,20.00,27.81,NaN,"LINESTRING (2701971.251 1264550.459, 2701866.5...",0,6916,115.886530,20.32,40,residential
"(0, 4321)",3639,7701053,9.81,19.62,9.81,19.62,9.81,19.62,9.81,19.62,...,16.00,19.62,NaN,"LINESTRING (2701971.251 1264550.459, 2701945.5...",0,4321,81.750330,16.06,40,residential
"(1, 57401)",3630,7715687,5.86,11.73,5.86,11.73,5.86,11.73,5.86,11.73,...,12.00,15.00,NaN,"LINESTRING (2699643.284 1265696.191, 2699595.2...",1,57401,48.861403,12.02,40,secondary
"(2, 57401)",3631,7715688,4.39,8.79,4.39,8.79,4.39,8.79,4.39,8.79,...,11.00,8.79,NaN,"LINESTRING (2699596.958 1265741.915, 2699595.2...",2,57401,36.619106,11.00,40,residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(1, 11923)",3629,7715686,9.62,19.24,9.62,19.24,9.62,19.24,9.62,19.24,...,16.00,19.24,NaN,"LINESTRING (2699643.284 1265696.191, 2699722.1...",1,11923,80.184124,15.59,40,secondary
"(61652, 39215)",75829,7591240,3.14,10.48,3.14,10.48,3.14,10.48,3.14,10.48,...,10.00,10.48,NaN,"LINESTRING (2699666.474 1236986.344, 2699708.9...",61652,39215,43.657291,10.02,40,tertiary
"(5, 30213)",3628,7701043,17.95,35.91,17.95,35.91,17.95,35.91,17.95,35.91,...,18.00,35.91,NaN,"LINESTRING (2702296.220 1264387.286, 2702405.5...",5,30213,140.285108,18.00,40,residential


In [8]:
tt['oneway'] = 'yes'

In [9]:
min(tt['length'])

1.04974657298051

In [10]:
G = snman.street_graph.street_graph_from_gdf(nd, tt)

In [11]:
snman.io.export_osm_xml(
    G,
    os.path.join(outputs_path, f'tt_car_{STATE}.osm'),
    tags=['LinkId', 'Freespeed', 'maxspeed', 'travel_time_s', 'length', 'highway', 'oneway'],
    uv_tags=True,
    overwrite_highway='primary',
    dont_overwrite_highway=['path'],
    set_maxspeed_by_cost='travel_time_s',
    raw_graph=True,
    floor_maxspeed=0.1,
    ceil_maxspeed=120
)

snman.io.osm_to_pbf(os.path.join(outputs_path, f'tt_car_{STATE}.osm'))

In [12]:
if 0:
    snman.io.export_street_graph(
        G,
        os.path.join(outputs_path, f'tt_{STATE}_edges.gpkg'),
        os.path.join(outputs_path, f'tt_{STATE}_nodes.gpkg'),
        crs=CRS_for_export
    )